## Train the model

In [1]:
from utils import dataloader
import utils.model
from datetime import datetime

filter_timestamp = cutoff_timestamp = None#int(datetime(2021, 2, 19, 0).timestamp())
use_user_index = "user_index.parquet"#"train_user_index.parquet"
csv_data_location = "downloaded_data"

In [2]:
dl = dataloader.RecSys2021TSVDataLoader(csv_data_location, use_user_index, mode="train", filter_timestamp=filter_timestamp)

In [3]:
%%time
data = next(iter(dl))
train_data = data[0]
labels = data[1]

CPU times: user 3min 9s, sys: 8.77 s, total: 3min 18s
Wall time: 3min 22s


In [4]:
recsysxgb = utils.model.RecSysXGB1()
xgb_params = {'objective': 'binary:logistic', 'eval_metric':'map'}
recsysxgb.train_in_memory(train_data, labels, xgb_params, save_dir = "xgb_models_05_submission")

## Evaluate the model

In [9]:
#recsysxgb = utils.model.RecSysXGB1("xgb_models_04_better_features1")
dl = dataloader.RecSys2021TSVDataLoader(csv_data_location, use_user_index, mode="val", filter_timestamp=filter_timestamp)

In [10]:
res = recsysxgb.evaluate_validation_set(dl)
for (target, ap, rce) in zip(recsysxgb.targets__, res[0], res[1]):
    print(f"{target}: {ap} - {rce}")

has_reply: 0.16962196226086176 - 10.747179145045871
has_retweet: 0.38669954726459954 - 19.86339784995025
has_retweet_comment: 0.043576703594668656 - -20.327808991302177
has_like: 0.6932498419910613 - 9.419793147661126


In [20]:
dict(sorted(recsysxgb.clfs_["has_like"].get_score(importance_type='gain').items(), key=lambda item: item[1],reverse=True))

{'TE_like_a_A': 17684.535543281632,
 'TE_retweet_b_B': 3859.1417743324137,
 'n_like_a_A': 3752.7517747058814,
 'type_encoding': 2855.152270493333,
 'n_like_b_B': 2604.5063219985723,
 'n_present_a_A': 2312.108605638519,
 'n_present_b_A': 1986.8852737500001,
 'a_follower_count': 1946.3226197428571,
 'TE_reply_b_B': 1858.9121773749998,
 'n_retweet_b_A': 1570.3047206769234,
 'bert_token_len': 1454.75,
 'n_photos': 1375.9783322888889,
 'TE_like_b_B': 1352.4801489709928,
 'b_creation_delta': 1246.4472216285715,
 'n_reply_a_A': 1227.1597588233333,
 'a_follows_b': 1008.1858988,
 'TE_retweet_a_B': 957.880188,
 'n_day_45_A': 826.6675516133333,
 'a_is_verified': 775.8146670000001,
 'n_retweet_comment_a_A': 723.8681656666668,
 'TE_retweet_a_A': 680.0563918333332,
 'n_day_40_A': 617.94104,
 'n_retweet_b_B': 520.4279785,
 'b_following_count': 452.9670416666666,
 'n_retweet_a_A': 255.85770425,
 'n_present_b_B': 232.654556,
 'n_day_36_B': 67.7659912,
 'TE_like_a_B': 26.8093872,
 'n_retweet_a_B': 24.71

## Try sample test run

In [ ]:
import utils.model
import utils.dataloader

dl = utils.dataloader.RecSys2021TSVDataLoader("test", "user_index.parquet", mode="test", load_n_batches=-1)
recsysxgb = utils.model.RecSysXGB1("xgb_models_05_submission")

recsysxgb.evaluate_test_set(testLoader = dl, output_file = "res.csv")

## Testing Custom batch sizes

In [2]:
from utils import dataloader
import utils.model
from datetime import datetime

filter_timestamp = cutoff_timestamp = None#int(datetime(2021, 2, 19, 0).timestamp())
use_user_index = "user_index.parquet"#"train_user_index.parquet"
csv_data_location = "downloaded_data"

In [3]:
dl = dataloader.RecSys2021TSVDataLoader(csv_data_location, use_user_index, mode="train", filter_timestamp=filter_timestamp, load_n_batches=1, batch_size=5000000, verbose=1)

In [4]:
r = [a[0] for a in dl]

Finished Batch Nr. 1 from file part-00001.tsv!


In [5]:
r[0]

,a_follower_count,a_following_count,a_is_verified,b_follower_count,b_following_count,b_is_verified,a_follows_b,bert_token_len,n_photos,n_videos,...,TE_like_b_A,TE_like_b_B,TE_retweet_a_A,TE_retweet_a_B,TE_retweet_b_A,TE_retweet_b_B,TE_retweet_comment_a_A,TE_retweet_comment_a_B,TE_retweet_comment_b_A,TE_retweet_comment_b_B
0,526918,163,False,220,638,False,False,3,0.693147,0.0,...,0.655940,0.582249,0.040542,0.050395,0.158352,0.006139,0.001334,0.005083,0.011672,0.000292
1,3062,1695,False,392,487,False,True,3,-1.000000,-1.0,...,0.718410,0.644231,0.020698,0.055194,0.173433,0.053160,0.019945,0.005567,0.012784,0.001226
2,2298,346,False,1306,5000,False,True,2,-1.000000,-1.0,...,0.285400,0.757480,0.035474,0.074269,0.031520,0.103158,0.000508,0.002088,0.007086,0.005966
3,4813450,662,True,90,91,False,False,3,0.693147,0.0,...,0.754331,0.705587,0.031772,0.052660,0.132105,0.148972,0.004921,0.002851,0.013423,0.012081
4,2740151,15255,True,158,411,False,False,3,-1.000000,-1.0,...,0.754331,0.803464,0.016482,0.057954,0.132105,0.105684,0.002477,0.005845,0.013423,0.010738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699225,1682398,4939,True,288,390,False,False,3,0.693147,0.0,...,0.754331,0.371785,0.026459,0.035124,0.132105,0.348737,0.007123,0.003542,0.013423,0.000831
1699226,138545,267,False,197,450,False,False,4,-1.000000,-1.0,...,0.699418,0.504699,0.176268,0.105287,0.114874,0.089691,0.005643,0.002386,0.011672,0.000226
1699227,4018,4048,False,112,85,False,True,3,-1.000000,-1.0,...,0.731210,0.763234,0.017553,0.022727,0.120095,0.036060,0.000950,0.002292,0.012203,0.002658
1699228,523356,6145,True,0,73,False,False,3,-1.000000,-1.0,...,0.753609,0.837485,0.095438,0.057954,0.110087,0.043313,0.006496,0.005845,0.011186,0.004401


small batch sizes are unproportionally slow. Probably the left merge with the index is the slow thing but not sure